In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Parametri
ticker_symbol = "^SPX"
target_expiration = "2026-12-18"  # Inserire una delle scadenze disponibili sotto
min_strength_pct = 0.01  # Mostra solo barre sopra al 1% del massimo
grouping_interval = 50 # Intervallo di raggruppamento per gli strike
max_distance_pct = 0.20 # Mostra solo strike entro il 20% dall'ultimo prezzo

# Download opzioni
ticker = yf.Ticker(ticker_symbol)
all_expirations = ticker.options

if target_expiration not in all_expirations:
    raise ValueError(f"La scadenza {target_expiration} non è disponibile. Disponibili: {all_expirations}")

opt_chain = ticker.option_chain(target_expiration)
calls = opt_chain.calls.copy()
puts = opt_chain.puts.copy()

# Calcolo forza relativa = open interest
calls["strength"] = calls["openInterest"]
puts["strength"] = puts["openInterest"]

# Raggruppamento per intervallo (tutte le opzioni vengono raggruppate qui)
calls["strike_group"] = (calls["strike"] // grouping_interval) * grouping_interval
puts["strike_group"] = (puts["strike"] // grouping_interval) * grouping_interval

grouped_calls = calls.groupby("strike_group")["strength"].sum().reset_index()
grouped_puts = puts.groupby("strike_group")["strength"].sum().reset_index()

# Merge affiancato su strike group
merged_grouped = pd.merge(
    grouped_calls.rename(columns={"strength": "call_strength", "strike_group": "strike"}),
    grouped_puts.rename(columns={"strength": "put_strength", "strike_group": "strike"}),
    on="strike",
    how="outer"
).fillna(0).sort_values("strike")

# Add distance filter: rimuove strike price groups outside the specified percentage distance from the current price
current_price = ticker.history(period="1d")["Close"].iloc[0]
merged_grouped["distance_pct"] = np.abs(merged_grouped["strike"] - current_price) / current_price
merged_grouped_distance_filtered = merged_grouped[merged_grouped["distance_pct"] <= max_distance_pct].copy()


# Calcola la forza totale del gruppo per il filtro
merged_grouped_distance_filtered["total_strength"] = merged_grouped_distance_filtered["call_strength"] + merged_grouped_distance_filtered["put_strength"]

# Filtro dinamico: rimuove gruppi con forza totale sotto soglia relativa (APPLICATO DOPO IL RAGGRUPPAMENTO)
max_total_strength = merged_grouped_distance_filtered["total_strength"].max()
merged_grouped_filtered = merged_grouped_distance_filtered[merged_grouped_distance_filtered["total_strength"] >= max_total_strength * min_strength_pct].copy()

# Calculate Net Open Interest
merged_grouped_filtered["net_oi"] = merged_grouped_filtered["call_strength"] - merged_grouped_filtered["put_strength"]


# Plot Open Interest (Calls vs Puts - Side-by-Side) - Reverted to side-by-side as requested
x = merged_grouped_filtered["strike"].astype(str)
call_heights = merged_grouped_filtered["call_strength"]
put_heights = merged_grouped_filtered["put_strength"]

bar_width = 0.4
x_indexes = np.arange(len(x)) # Use numpy for easier index handling

plt.figure(figsize=(14, 6))
plt.bar(x_indexes - bar_width/2, call_heights, width=bar_width, label="CALL OI", align="center")
plt.bar(x_indexes + bar_width/2, put_heights, width=bar_width, label="PUT OI", align="center")
plt.xticks(x_indexes, x, rotation=90)
plt.xlabel("Strike Price Group")
plt.ylabel("Open Interest")
plt.title(f"{ticker_symbol} DPD - Exp: {target_expiration} (Grouped by {grouping_interval}, Filtered by Total Strength and Distance)")
plt.legend()
plt.tight_layout()
plt.grid(True)

# Add vertical line for current price
closest_strike_group_index = np.abs(merged_grouped_filtered["strike"] - current_price).argmin()
plt.axvline(x=x_indexes[closest_strike_group_index], color='red', linestyle='--', label=f'Current Price: {current_price:.2f}')
plt.legend()

plt.show()

# Plot Net Open Interest (All bars above X-axis)
plt.figure(figsize=(14, 6))
# Plot the absolute value of net_oi, using color to indicate original sign
plt.bar(x_indexes, merged_grouped_filtered["net_oi"].abs(), width=0.8, label="Net OI (Call - Put)", align="center", color=np.where(merged_grouped_filtered["net_oi"] >= 0, 'green', 'red'))
plt.xticks(x_indexes, x, rotation=90)
plt.xlabel("Strike Price Group")
plt.ylabel("Absolute Net Open Interest") # Changed label to reflect absolute value
plt.title(f"{ticker_symbol} DPD - Net Open Interest - Exp: {target_expiration} (Grouped by {grouping_interval}, Filtered by Total Strength and Distance)")
plt.legend()
plt.tight_layout()
plt.grid(True)

# Add vertical line for current price
plt.axvline(x=x_indexes[closest_strike_group_index], color='red', linestyle='--', label=f'Current Price: {current_price:.2f}')
plt.legend()

plt.show()


# Display the filtered grouped data table with Net OI
display(merged_grouped_filtered)